In [2]:
#libraries
import os
from pathlib import Path
os.chdir(Path(os.getcwd()).parents[0]) 

In [3]:
pwd

'/mnt/c/Users/maxpu/Dropbox/scripts/Projects/abstraction'

In [4]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import torch
# from STOMPnet import STOMPnet,Decoder
# from STOMPnet2 import STOMPnet2
# from utils import MultiChannelNet, get_width, count_parameters
from models import get_width
import h5py
import yaml
import json
import hashlib
import wandb

In [6]:
def get_runs():
    api = wandb.Api()
    runs = api.runs("TOMMM")
    summary_list, config_list, name_list, state_list = [], [], [], []
    for run in runs:
        # .summary contains output keys/values for
        # metrics such as accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append({k: v for k, v in run.config.items()}) # if not k.startswith("_")})

        # .name is the human-readable name of the run.
        name_list.append(run.name)

        state_list.append(run.state)

    runs_df = pd.DataFrame(
        {"summary": summary_list, "config": config_list, "name": name_list, "state": state_list}
    )

#     runs_df.to_csv("project.csv")
#     runs_df.loc[:,['state','config']].apply(lambda run: run.state+
#                                            ': N'+str(run.config['num_agents'])+
#                                            ', P'+str(run.config['P'])+
#                                            ' '+run.config['model_name'] if run.state=='finished' else run.state,axis=1)
    return runs_df
runs_df=get_runs()

{'parameters/seq_enc.ISAB.mab1.fc_k.weight': {'_type': 'histogram',
  'bins': [-0.07761244475841522,
   -0.07518714666366577,
   -0.07276184856891632,
   -0.07033655792474747,
   -0.06791125982999802,
   -0.06548596173524857,
   -0.06306066364049911,
   -0.06063536927103996,
   -0.05821007117629051,
   -0.05578477308154106,
   -0.05335947871208191,
   -0.05093418061733246,
   -0.04850888252258301,
   -0.046083588153123856,
   -0.043658290058374405,
   -0.04123299568891525,
   -0.0388076975941658,
   -0.03638239949941635,
   -0.0339571051299572,
   -0.03153180703520775,
   -0.029106510803103447,
   -0.026681214570999146,
   -0.024255916476249695,
   -0.021830620244145393,
   -0.019405324012041092,
   -0.01698002777993679,
   -0.014554730616509914,
   -0.012129433453083038,
   -0.009704137220978737,
   -0.007278840523213148,
   -0.004853543825447559,
   -0.0024282471276819706,
   -2.950429916381836e-06,
   0.002422346267849207,
   0.004847642965614796,
   0.0072729396633803844,
   0.0096